<a href="https://colab.research.google.com/github/jana-nf/Estagio_CBL_CienciaDeDados_GovernancaDadosHDL_AuditoriaGit/blob/main/Git_Avancado_Governanca_Dados.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#GIT AVANÇADO PARA GOVERNANÇA DE DADOS

###Apresento um "Cenário de Laboratório" perfeito, desde a criação de um ambiente completo aqui no Google Colab, até a simulação de um histórico de commits onde um erro é introduzido "silenciosamente", permitindo a execução do ***git bisect*** na prática.

🛠️ Inicialização do Ambiente e Histórico

Configurar o Git e criar as camadas Bronze e Silver com um erro oculto no meio dos commits.

In [1]:
# 1. Configuração de Identidade
!git config --global user.email "analista@empresa.com"
!git config --global user.name "Analista de Dados"

# 2. Criar diretórios do Lakehouse
!mkdir -p data/bronze data/silver tests
!git init

# 3. Commit Inicial (Versão Estável v1.0.0)
with open('data/silver/transform.py', 'w') as f:
    f.write("tax_rate = 0.15\nprint(f'Taxa atual: {tax_rate}')")
!git add .
!git commit -m "v1.0.0 - Release estável da camada Silver"
!git tag v1.0.0

# 4. Sequência de Commits (Simulando desenvolvimento)
for i in range(1, 4):
    with open(f'feature_{i}.txt', 'w') as f: f.write(f"Feature {i}")
    !git add .
    !git commit -m "feat: incremento de pipeline {i}"

# 5. O COMMIT DO BUG (Simulando erro de digitação de 0.15 para 0.05)
with open('data/silver/transform.py', 'w') as f:
    f.write("tax_rate = 0.05\nprint(f'Taxa atual: {tax_rate}')")
!git add .
!git commit -m "fix: pequenos ajustes na lógica de transformação"

# 6. Mais commits após o erro
for i in range(4, 6):
    with open(f'feature_{i}.txt', 'w') as f: f.write(f"Feature {i}")
    !git add .
    !git commit -m "feat: incremento de pipeline {i}"

print("\n✅ Ambiente pronto! O bug foi introduzido em algum lugar do histórico.")

hint: Using 'master' as the name for the initial branch. This default branch name
hint: is subject to change. To configure the initial branch name to use in all
hint: of your new repositories, which will suppress this warning, call:
hint: 
hint: 	git config --global init.defaultBranch <name>
hint: 
hint: Names commonly chosen instead of 'master' are 'main', 'trunk' and
hint: 'development'. The just-created branch can be renamed via this command:
hint: 
hint: 	git branch -m <name>
Initialized empty Git repository in /content/.git/
[master (root-commit) 6815462] v1.0.0 - Release estável da camada Silver
 22 files changed, 51027 insertions(+)
 create mode 100644 .config/.last_opt_in_prompt.yaml
 create mode 100644 .config/.last_survey_prompt.yaml
 create mode 100644 .config/.last_update_check.json
 create mode 100644 .config/active_config
 create mode 100644 .config/config_sentinel
 create mode 100644 .config/configurations/config_default
 create mode 100644 .config/default_configs.db
 cr

🧪 Criar o Script de Validação

Este é o "juiz" que dirá ao Git se o commit está bom ou ruim.

In [2]:
%%writefile tests/validate_metrics.py
import sys

try:
    with open('data/silver/transform.py', 'r') as f:
        content = f.read()
        # Se a taxa não for 0.15, o teste falha
        if "tax_rate = 0.15" in content:
            sys.exit(0) # Sucesso
        else:
            sys.exit(1) # Falha
except Exception:
    sys.exit(1)

Writing tests/validate_metrics.py


🔍 Executar a Auditoria (Git Bisect)

Rodar o comando que automatiza a busca pelo erro.

In [3]:
# Iniciando a busca binária
print("Iniciando Bisect Automático...\n")
!git bisect start HEAD v1.0.0
!git bisect run python tests/validate_metrics.py

print("\n🎯 O Git encontrou o commit culpado acima!")

Iniciando Bisect Automático...

Bisecting: 2 revisions left to test after this (roughly 2 steps)
[8e16746ad4ab81bca4f5e6a61983340bac6ac9e8] feat: incremento de pipeline 3
running  'python' 'tests/validate_metrics.py'
Bisecting: 0 revisions left to test after this (roughly 1 step)
[3b2025e2a04687b4f1bbd3071a4e352eb898c6bf] feat: incremento de pipeline 4
running  'python' 'tests/validate_metrics.py'
Bisecting: 0 revisions left to test after this (roughly 0 steps)
[79893ed2db9afe7d4a72f9d159982d7ace6beeae] fix: pequenos ajustes na lógica de transformação
running  'python' 'tests/validate_metrics.py'
79893ed2db9afe7d4a72f9d159982d7ace6beeae is the first bad commit
commit 79893ed2db9afe7d4a72f9d159982d7ace6beeae
Author: Analista de Dados <analista@empresa.com>
Date:   Fri Jan 2 16:30:45 2026 +0000

    fix: pequenos ajustes na lógica de transformação

 data/silver/transform.py | 2 +-
 1 file changed, 1 insertion(+), 1 deletion(-)
bisect found first bad commit
🎯 O Git encontrou o commit culp

🔄 O Rollback Profissional

Após identificar o "culpado", executar o revert para criar o commit de correção auditável.

###Dica:

"Em um ambiente de produção e governança de dados, a imutabilidade do histórico é fundamental. O git reset apaga commits, o que destrói a trilha de auditoria. Já o git revert cria um novo commit que desfaz a alteração anterior. Isso permite que qualquer auditor veja exatamente quando o erro foi introduzido, quem o identificou e como ele foi corrigido, mantendo a transparência total do pipeline."

In [4]:
# Substitua pelo SHA que o bisect encontrou (ou use o ponteiro do bisect)
!git revert bisect/bad --no-edit
!git bisect reset

print("\n✅ Rollback concluído! O histórico foi preservado e o erro corrigido.")
!git log --oneline -n 5

[detached HEAD 6cc189b] Revert "fix: pequenos ajustes na lógica de transformação"
 Date: Fri Jan 2 16:30:46 2026 +0000
 1 file changed, 1 insertion(+), 1 deletion(-)
any of your branches:

  6cc189b Revert "fix: pequenos ajustes na lógica de transformação"

If you want to keep it by creating a new branch, this may be a good time
to do so with:

 git branch <new-branch-name> 6cc189b

Switched to branch 'master'

✅ Rollback concluído! O histórico foi preservado e o erro corrigido.
350b435 (HEAD -> master) feat: incremento de pipeline 5
3b2025e feat: incremento de pipeline 4
79893ed fix: pequenos ajustes na lógica de transformação
8e16746 feat: incremento de pipeline 3
d4e9157 feat: incremento de pipeline 2


####Python/Bash

In [7]:
import os

# Create directory if it doesn't exist
os.makedirs('05_optimization', exist_ok=True)

# Content to write
file_content = """def check_optimization():
    print("📊 Relatório de Otimização de Assets (Fase 5)")

    # Verifica se o arquivo .gitattributes existe (prova de LFS)
    if os.path.exists('../.gitattributes'):
        print("- Git LFS: Configurado [OK]")

    # Verifica presença de submódulos
    if os.path.exists('sql_utils'):
        print("- Submódulos PL/SQL: Integrados [OK]")

    print("- Estratégia Sparse-Checkout: Disponível em setup_gold_only.sh")

if __name__ == "__main__":
    check_optimization()
"""

# Write to file
with open('05_optimization/asset_report.py', 'w') as f:
    f.write(file_content)

print("Writing 05_optimization/asset_report.py")

Writing 05_optimization/asset_report.py


📊 Ao rodar esse projeto no Colab, provo que:

Independente de máquina, o pipeline de governança funciona em qualquer ambiente Linux/Cloud.

Para automatizar, você não precisa testar manualmente, pois o código testa para você.


A tabela abaixo é o fechamento estratégico pois demonstra que não apenas conhecimento técnico, mas um nível estratégico que pensa em custos e escalabilidade (FinOps).

Na de Integração e Otimização, o grande dilema é:

***onde guardar o quê?***

Colocar tudo no Git (mesmo com LFS) pode ser caro e lento, enquanto colocar tudo no Banco de Dados pode dificultar o versionamento de modelos e dashboards.

📊 Estratégias de Armazenamento e Governança
Esta tabela resume os critérios de decisão para o um pipeline HDL (Hybrid Data Lakehouse):

### 📊 Matriz de Decisão: Estratégia de Armazenamento (Fase 5)

| Critério | Git Padrão (Code) | Git LFS (Large Files) | Banco de Dados (SQL/NoSQL) | Cloud Storage (S3/Blob) |
| :--- | :--- | :--- | :--- | :--- |
| **Tipo de Asset** | Scripts, YAML, Configs | Dashboards, Modelos ML | Tabelas, Views, Metadata | Dados Brutos (Bronze) |
| **Custo ($)** | Baixo | Médio (Bandwidth) | Alto (Instância/Query) | Muito Baixo |
| **Versionamento** | Granular (Linha) | Snapshot (Binário) | Complexo (Requer DDL) | Via Metadados |
| **Performance** | Instantânea | Lenta no Clone | Rápida para Queries | Alta Ingestão |
| **Exemplo no CBL** | `transform.py` | `model_v1.onnx` | `vendas_gold_view` | `raw_data_2026.csv` |